In [ ]:
import psycopg2
import csv
import folium


In [ ]:
import pandas as pd

file_path = 'crimedatadenver.csv'
crime_data = pd.read_csv(file_path)

columns_to_remove = [
    'OBJECTID', 'INCIDENT_ID', 'OFFENSE_ID', 
    'OFFENSE_CODE', 'OFFENSE_CODE_EXTENSION', 'DISTRICT_ID',
    'LAST_OCCURRENCE_DATE', 'REPORTED_DATE', 'GEO_X', 'GEO_Y',
    'NEIGHBORHOOD_ID', 'IS_CRIME', 'IS_TRAFFIC', 'VICTIM_COUNT', 'x', 'y'
]

crime_data_cleaned = crime_data.drop(columns=columns_to_remove)

crime_data_cleaned.head(20)

In [ ]:
crime_data_cleaned.to_csv("cleaned_crime_data.csv", index=False)

In [ ]:
import sqlite3
import pandas as pd

file_path = 'cleaned_crime_data.csv'
crime_data = pd.read_csv(file_path)

db_path = 'crime_data.db'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute('DROP TABLE IF EXISTS denver_crime_data')

cursor.execute('''
CREATE TABLE denver_crime_data (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    latitude REAL,
    longitude REAL,
    address TEXT,
    crime_type TEXT,
    precinct INTEGER,
    first_occurrence_date TEXT
)
''')

for _, row in crime_data.iterrows():
    cursor.execute('''
    INSERT INTO denver_crime_data (latitude, longitude, address, crime_type, precinct, first_occurrence_date)
    VALUES (?, ?, ?, ?, ?, ?)
    ''', (
        row['GEO_LAT'],
        row['GEO_LON'],
        row.get('address', ''),
        row['OFFENSE_TYPE_ID'],
        row['PRECINCT_ID'],
        row.get('FIRST_OCCURRENCE_DATE', '')
    ))

conn.commit()
conn.close()

print("Data successfully inserted into the database.")

In [ ]:
import sqlite3
import pandas as pd
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
import numpy as np
import folium
from folium.plugins import FastMarkerCluster
from datetime import datetime, timedelta

db_path = 'crime_data.db'

conn = sqlite3.connect(db_path)
crime_df = pd.read_sql_query(
    "SELECT rowid, latitude, longitude, first_occurrence_date FROM denver_crime_data", conn
)
conn.close()

print(crime_df.head())

crime_df['first_occurrence_date'] = pd.to_datetime(crime_df['first_occurrence_date'], errors='coerce')

crime_df = crime_df.dropna(subset=['first_occurrence_date'])

two_years_ago = datetime.now() - timedelta(days=2 * 365)

crime_df = crime_df[crime_df['first_occurrence_date'] >= two_years_ago]

crime_df = crime_df.dropna(subset=['latitude', 'longitude'])

crime_df = crime_df.drop_duplicates()

crime_df = crime_df.head(10000)

coordinates = crime_df[['latitude', 'longitude']].to_numpy()

scaler = StandardScaler()
coordinates_scaled = scaler.fit_transform(coordinates)

# Perform DBSCAN clustering
best_eps = 0.05
best_min_samples = 5

db = DBSCAN(eps=best_eps, min_samples=best_min_samples, n_jobs=-1).fit(coordinates_scaled)

crime_df['cluster'] = db.labels_

print(f"Number of clusters found: {len(set(db.labels_)) - (1 if -1 in db.labels_ else 0)}")

conn = sqlite3.connect(db_path)

cursor = conn.cursor()
cursor.execute("PRAGMA table_info(denver_crime_data);")
columns = [column[1] for column in cursor.fetchall()]

if 'cluster' not in columns:
    conn.execute("ALTER TABLE denver_crime_data ADD COLUMN cluster INTEGER;")
    print("Added 'cluster' column to the table.")
else:
    print("'cluster' column already exists in the table.")

for index, row in crime_df.iterrows():
    conn.execute(
        "UPDATE denver_crime_data SET cluster = ? WHERE rowid = ?",
        (int(row['cluster']), int(row['id']))
    )

conn.commit()
conn.close()

print("Database updated with cluster labels.")

# Initialize the map centered around the mean coordinates
map_center = [crime_df['latitude'].mean(), crime_df['longitude'].mean()]
crime_map = folium.Map(location=map_center, zoom_start=12)

# Use FastMarkerCluster for efficient marker visualization
marker_data = [
    (row['latitude'], row['longitude'])
    for _, row in crime_df[crime_df['cluster'] != -1].iterrows()
]
FastMarkerCluster(marker_data).add_to(crime_map)

crime_map.save("denver_crime_clusters.html")
print("Cluster map saved as denver_crime_clusters.html.")


In [ ]:
import sqlite3
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import folium

db_path = 'crime_data.db'

conn = sqlite3.connect(db_path)
crime_df = pd.read_sql_query(
    "SELECT cluster, latitude, longitude, first_occurrence_date FROM denver_crime_data WHERE cluster IS NOT NULL AND cluster != -1",
    conn
)
conn.close()

crime_df['first_occurrence_date'] = pd.to_datetime(crime_df['first_occurrence_date'], errors='coerce')

crime_df = crime_df.dropna(subset=['first_occurrence_date'])

unique_clusters = crime_df['cluster'].unique()

cluster_forecasts = {}
cluster_centroids = {}

for cluster_id in unique_clusters:
    cluster_data = crime_df[crime_df['cluster'] == cluster_id]

    time_series = (
        cluster_data
        .set_index('first_occurrence_date')
        .resample('D')
        .size()
    )

    # Handle cases where time series is too short for ARIMA
    if len(time_series) < 5:
        print(f"Cluster {cluster_id} skipped due to insufficient data.")
        continue

    # Fit ARIMA model
    try:
        model = ARIMA(time_series, order=(1, 1, 1))
        model_fit = model.fit()
       
        # Forecast the next 30 days
        forecast = model_fit.forecast(steps=30)
        cluster_forecasts[cluster_id] = forecast
       
        # Calculate the centroid for the cluster
        cluster_centroids[cluster_id] = [
            cluster_data['latitude'].mean(),
            cluster_data['longitude'].mean()
        ]
    except Exception as e:
        print(f"ARIMA failed for cluster {cluster_id}: {e}")

crime_map = folium.Map(location=[39.7392, -104.9903], zoom_start=12)

# Add markers for each cluster with forecasted data
for cluster_id, forecast in cluster_forecasts.items():
    avg_forecast = forecast.mean()

    # Determine marker size and color based on severity
    size = 7 + (avg_forecast * 0.2)
    size = min(max(size, 7), 10)
    if avg_forecast > 10:
        color = 'red'  # High risk
    elif avg_forecast > 3:
        color = 'orange'  # Medium risk
    else:
        color = 'green'  # Low risk

    # Add marker to the map
    folium.CircleMarker(
        location=cluster_centroids[cluster_id],
        radius=size,
        color=color,
        fill=True,
        fill_opacity=0.7,
        popup=folium.Popup(
            f"Hotspot {cluster_id}<br>Avg Forecast: {avg_forecast:.2f}<br>Forecast Values: {list(forecast)}",
            max_width=300
        )
    ).add_to(crime_map)

crime_map.save("denver_crime_forecast_map.html")
print("Forecast map saved as denver_crime_forecast_map.html.")

In [ ]:
import sqlite3
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

db_path = 'crime_data.db'

conn = sqlite3.connect(db_path)
crime_data = pd.read_sql_query(
    "SELECT cluster, first_occurrence_date FROM denver_crime_data WHERE cluster IS NOT NULL AND cluster != -1",
    conn
)
conn.close()

crime_data['first_occurrence_date'] = pd.to_datetime(crime_data['first_occurrence_date'], errors='coerce')

crime_data = crime_data.dropna(subset=['first_occurrence_date'])

cluster_forecasts = {}
evaluation_results = {}

unique_clusters = crime_data['cluster'].unique()

for cluster_id in unique_clusters:
    cluster_data = crime_data[crime_data['cluster'] == cluster_id]

    time_series = (
        cluster_data
        .set_index('first_occurrence_date')
        .resample('D')
        .size()
    )

    if len(time_series) < 10:
        print(f"Cluster {cluster_id} skipped due to insufficient data.")
        continue

    split_point = int(len(time_series) * 0.8)  # 80% for training, 20% for testing
    train_data = time_series[:split_point]
    test_data = time_series[split_point:]

    try:
        model = ARIMA(train_data, order=(1, 1, 1))  # Adjust (p, d, q) if needed
        model_fit = model.fit()

        forecast_steps = len(test_data)
        forecast = model_fit.forecast(steps=forecast_steps)

        mae = mean_absolute_error(test_data, forecast)
        rmse = np.sqrt(mean_squared_error(test_data, forecast))

        cluster_forecasts[cluster_id] = forecast
        evaluation_results[cluster_id] = {'MAE': mae, 'RMSE': rmse}

        print(f"Cluster {cluster_id}: MAE = {mae:.2f}, RMSE = {rmse:.2f}")
    except Exception as e:
        print(f"ARIMA failed for cluster {cluster_id}: {e}")


evaluation_df = pd.DataFrame.from_dict(evaluation_results, orient='index')
evaluation_df.reset_index(inplace=True)
evaluation_df.rename(columns={'index': 'cluster'}, inplace=True)

evaluation_df.to_csv("model_evaluation_results.csv", index=False)
print("Evaluation results saved to 'model_evaluation_results.csv'.")